In [19]:
import networkx as nx
import numpy as  np
import random
import json
from time import process_time
from functools import partial
import concurrent.futures
from threading import Thread

## Carregando substrato e requisições


In [20]:
G = nx.read_gml("infra.gml", label="id")

with open('vnr.json', 'r') as f:
    vnrs = json.load(f)
    
with open('populacao_inical.json', 'r') as f:
    populacao_inicial = json.load(f)

In [21]:
_, max_cpu = max(G.nodes.data('cpu'), key=lambda t: t[1])
_, _, max_bandwidth = max(G.edges.data('bandwidth'), key=lambda t: t[2])

In [22]:
max_cpu, max_bandwidth

(98, 9938)

In [23]:
_, min_cpu = min(G.nodes.data('cpu'), key=lambda t: t[1])
_, _, min_bandwidth = min(G.edges.data('bandwidth'), key=lambda t: t[2])

In [24]:
min_cpu, min_bandwidth

(11, 120)

In [25]:
populacao_inicial

[{'nos': [34, 40, 41]},
 {'nos': [76, 45, 106]},
 {'nos': [80, 100, 105]},
 {'nos': [14, 91, 37]},
 {'nos': [25, 55, 65]},
 {'nos': [61, 30, 10]},
 {'nos': [76, 5, 106]},
 {'nos': [68, 12, 39]},
 {'nos': [66, 80, 45]},
 {'nos': [7, 80, 95]},
 {'nos': [78, 22, 38]},
 {'nos': [58, 42, 68]},
 {'nos': [89, 88, 78]},
 {'nos': [24, 57, 58]},
 {'nos': [36, 10, 72]},
 {'nos': [75, 28, 13]},
 {'nos': [99, 1, 14]},
 {'nos': [53, 95, 96]},
 {'nos': [46, 0, 11]},
 {'nos': [21, 102, 74]}]

In [26]:
vnrs

[{'cpu': 42, 'bandwidth': 1044, 'qtd_nos': 3, 'topologia': [[1, 2]]},
 {'cpu': 72,
  'bandwidth': 7103,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 17,
  'bandwidth': 949,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 44,
  'bandwidth': 1272,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 89, 'bandwidth': 5621, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]},
 {'cpu': 41, 'bandwidth': 5263, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2]]},
 {'cpu': 76, 'bandwidth': 0, 'qtd_nos': 3, 'topologia': []},
 {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]},
 {'cpu': 59, 'bandwidth': 3010, 'qtd_nos': 3, 'topologia': [[0, 1], [1, 2]]},
 {'cpu': 40, 'bandwidth': 2407, 'qtd_nos': 3, 'topologia': [[1, 2]]}]

## Filtrando e ordenando requisições

In [27]:
new_vnrs = list(filter(lambda VNR: VNR['qtd_nos'] < G.number_of_nodes(), vnrs))

In [28]:
new_vnrs = sorted(new_vnrs, key=lambda VNR: VNR['cpu']/max_cpu + VNR['bandwidth']/max_bandwidth)

In [29]:
new_vnrs

[{'cpu': 17,
  'bandwidth': 949,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]},
 {'cpu': 42, 'bandwidth': 1044, 'qtd_nos': 3, 'topologia': [[1, 2]]},
 {'cpu': 44,
  'bandwidth': 1272,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 40, 'bandwidth': 2407, 'qtd_nos': 3, 'topologia': [[1, 2]]},
 {'cpu': 76, 'bandwidth': 0, 'qtd_nos': 3, 'topologia': []},
 {'cpu': 59, 'bandwidth': 3010, 'qtd_nos': 3, 'topologia': [[0, 1], [1, 2]]},
 {'cpu': 41, 'bandwidth': 5263, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2]]},
 {'cpu': 72,
  'bandwidth': 7103,
  'qtd_nos': 3,
  'topologia': [[0, 1], [0, 2], [1, 2]]},
 {'cpu': 89, 'bandwidth': 5621, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]}]

## Atendendo as requisições

### Sequencial

In [30]:
class AlgoritmoGenetico():
    
    def __init__(self, substrato, VNR, populacao_inicial):
        self._substrato = substrato
        self._VNR = VNR
        _, self._max_cpu = max(substrato.nodes.data('cpu'), key=lambda t: t[1])
        _, _, self._max_bandwidth = max(substrato.edges.data('bandwidth'), key=lambda t: t[2])     
        self._populacao_inicial = populacao_inicial
    
    @property
    def VNR(self):
        return self._VNR
    
    @VNR.setter
    def VNR(self, new_VNR):
        self._VNR = new_VNR
        
    @property
    def substrato(self):
        return self._substrato
    
    @substrato.setter
    def substrato(self, new_substrato):
        self._substrato = new_substrato
        
    def atualiza_individuo(self, individuo):
        individuo['cpu'] = min([self.substrato.nodes[no]['cpu'] for no in individuo['nos']]) 
    
        try:
            bandwidths = []

            for no1, no2 in self._VNR['topologia']:
                caminhos = nx.dijkstra_path(self.substrato, individuo['nos'][no1], individuo['nos'][no2])
                bandwidths += [self.substrato.edges[caminhos[i], no]['bandwidth'] for i, no in enumerate(caminhos[1:])]
            
            individuo['bandwidth'] = min(bandwidths)
            individuo['qtd_saltos'] = len(bandwidths)
        except:
            individuo['bandwidth'] = 0
            individuo['qtd_saltos'] = 0
            
        return individuo
    
    def fitness(self, VNR):
        cpu = (VNR['cpu'] - self._VNR['cpu']) / self._max_cpu
        bandwidth = (VNR['bandwidth'] - self._VNR['bandwidth']) / self._max_bandwidth
        saltos = (VNR['qtd_saltos'] - len(self._VNR['topologia'])) / self.substrato.number_of_edges() 
        
        if cpu >= 0 and bandwidth < 0:
            return bandwidth
        elif cpu < 0 and bandwidth >= 0:
            return cpu

        return cpu + bandwidth - saltos
    
    def calcular_pontuacao(self, populacao):
        return [self.fitness(x) for x in populacao]
    
    def calculo_roleta(self, pontuacoes):
        menor_pontuacao = min(pontuacoes)
        
        pontuacoes_positivadas = list(map(lambda pontuacao: pontuacao-menor_pontuacao, pontuacoes))
        
        maior_pontuacao = max(pontuacoes_positivadas)
        
        pontuacoes_positivadas_reverso = list(map(lambda pontuacao: (maior_pontuacao-pontuacao) * 5, pontuacoes_positivadas))

        roleta = []
        total = sum(pontuacoes_positivadas_reverso)
        for i, valor in enumerate(pontuacoes_positivadas_reverso):
            repeticao = round(abs((total+1)/(valor + 1)))
            
            roleta.extend([i]*repeticao)
      
        return roleta
    
    def crossover(self, populacao, g1, g2):
        # {'nos': [109, 105, 66], 'cpu': 0, 'bandwidth': 100, 'qtd_saltos': 5}
        qtd_nos = len(populacao[g1]['nos'])
        
        new_individuo = {'nos': [-1 for _ in range(qtd_nos)]}
        
        metade = int(np.ceil(qtd_nos/2))
        
        indices_dos_nos = list(range(qtd_nos))
        random.shuffle(indices_dos_nos)    
        
        if random.random() < 0.5:
            g1, g2 = g2, g1
        
        g = g1
        for i, indice in enumerate(indices_dos_nos):
            if i == metade:
                g = g2
                
            if populacao[g]['nos'][indice] not in new_individuo['nos']:
                new_individuo['nos'][indice] = populacao[g]['nos'][indice]
            else:
                nos_possiveis = list(set(range(self.substrato.number_of_nodes())).difference(set(new_individuo['nos'])))
                new_individuo['nos'][indice] = random.choice(nos_possiveis)

        return self.atualiza_individuo(new_individuo)
    
    def melhor_individuo(self, populacao):
            pontuacoes = self.calcular_pontuacao(populacao)
                  
            indice_maior_pontuacao = pontuacoes.index(max(pontuacoes))
            
            return populacao[indice_maior_pontuacao]
            
    def fit(self, repeticoes=100, prob_mutacao=0.1, verbose=False):
        populacao = list(map(self.atualiza_individuo, self._populacao_inicial.copy()))
            
        tam_pop = len(populacao) 

        for count in range(repeticoes):            
            if verbose:
                print("Repetição", count)
            
            pontuacoes = self.calcular_pontuacao(populacao)
            indice_maior_pontuacao = pontuacoes.index(max(pontuacoes))
            if verbose:
                print('MELHOR INDIVIDUO DA EPOCA ', populacao[indice_maior_pontuacao], 'FITNESS: ', pontuacoes[indice_maior_pontuacao])
            
            roleta = self.calculo_roleta(pontuacoes)
            
            filhos = [populacao[indice_maior_pontuacao]]
            for tam in range(tam_pop-1):
                genitor1 = random.choice(roleta)
                genitor2 = random.choice(roleta)
                filho = self.crossover(populacao, genitor1, genitor2)

                if (random.random() < prob_mutacao):
                    qtd_nos_individuo = len(filho['nos'])
                    qtd_nos_substrato = self.substrato.number_of_nodes()
                    
                    indice_no = random.choice(range(qtd_nos_individuo))
                    
                    new_nos_possiveis = list(set(range(qtd_nos_substrato)).difference(set(filho['nos'])))
                        
                    filho['nos'][indice_no] = random.choice(new_nos_possiveis)

                    filho = self.atualiza_individuo(filho)
                    
                filhos.append(filho)
                        
            populacao = np.array(filhos) 
    
        return self.melhor_individuo(filhos), populacao
    
    def mapear_rede_virtual(self, request):
        print('Virtual Network requisitada: ', request, '\n')

        self.VNR = request

        melhor_individuo, populacao = self.fit(repeticoes=2000, prob_mutacao=0.2)

        return (melhor_individuo, request)
    
    def rede_virtual_valida(self, melhor_individuo, request):
        if (melhor_individuo['cpu'] >= request['cpu']) and (melhor_individuo['bandwidth'] >= request['bandwidth']):
            for no in melhor_individuo['nos']:
                self.substrato.nodes[no]['cpu'] -= request['cpu']

            for no1, no2 in request['topologia']:
                caminho = nx.dijkstra_path(self.substrato, melhor_individuo['nos'][no1], melhor_individuo['nos'][no2])

                for i, no in enumerate(caminho[1:]):
                    self.substrato.edges[(caminho[i], no)]['bandwidth'] -= request['bandwidth']

            print('Virtual Network mapeada: ', melhor_individuo, 'para a VNR: ', request, '\n')
        else:
            print('Não encontrou uma solução para a VNR ', request, '\n')

In [31]:
alg_genet = AlgoritmoGenetico(G.copy(), None, populacao_inicial)

In [32]:
t1 = process_time()

for request in new_vnrs:
    melhor_individuo, _ = alg_genet.mapear_rede_virtual(request)
    alg_genet.rede_virtual_valida(melhor_individuo, request)
    
t2 = process_time()

print('Tempo decorrido em segundos: ', t2-t1)

Virtual Network requisitada:  {'cpu': 17, 'bandwidth': 949, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2], [1, 2]]} 

Virtual Network mapeada:  {'nos': [40, 104, 37], 'cpu': 70, 'bandwidth': 6553, 'qtd_saltos': 4} para a VNR:  {'cpu': 17, 'bandwidth': 949, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2], [1, 2]]} 

Virtual Network requisitada:  {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]} 

Virtual Network mapeada:  {'nos': [85, 37, 98], 'cpu': 60, 'bandwidth': 6604, 'qtd_saltos': 3} para a VNR:  {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]} 

Virtual Network requisitada:  {'cpu': 42, 'bandwidth': 1044, 'qtd_nos': 3, 'topologia': [[1, 2]]} 

Virtual Network mapeada:  {'nos': [53, 6, 11], 'cpu': 97, 'bandwidth': 7209, 'qtd_saltos': 1} para a VNR:  {'cpu': 42, 'bandwidth': 1044, 'qtd_nos': 3, 'topologia': [[1, 2]]} 

Virtual Network requisitada:  {'cpu': 44, 'bandwidth': 1272, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2], [1, 2]]} 

Virtua

### Paralelo

In [33]:
class AlgoritmoGeneticoParalelo():
    
    def __init__(self, substrato, VNR, populacao_inicial):
        self._substrato = substrato
        self._VNR = VNR
        _, self._max_cpu = max(substrato.nodes.data('cpu'), key=lambda t: t[1])
        _, _, self._max_bandwidth = max(substrato.edges.data('bandwidth'), key=lambda t: t[2])     
        self._populacao_inicial = populacao_inicial
    
    @property
    def VNR(self):
        return self._VNR
    
    @VNR.setter
    def VNR(self, new_VNR):
        self._VNR = new_VNR
        
    @property
    def substrato(self):
        return self._substrato
    
    @substrato.setter
    def substrato(self, new_substrato):
        self._substrato = new_substrato
        
    def atualiza_individuo(self, individuo):
        individuo['cpu'] = min([self.substrato.nodes[no]['cpu'] for no in individuo['nos']]) 
    
        try:
            bandwidths = []

            for no1, no2 in self._VNR['topologia']:
                caminhos = nx.dijkstra_path(self.substrato, individuo['nos'][no1], individuo['nos'][no2])
                bandwidths += [self.substrato.edges[caminhos[i], no]['bandwidth'] for i, no in enumerate(caminhos[1:])]
            
            individuo['bandwidth'] = min(bandwidths)
            individuo['qtd_saltos'] = len(bandwidths)
        except:
            individuo['bandwidth'] = 0
            individuo['qtd_saltos'] = 0
            
        return individuo
    
    def fitness(self, VNR):
        cpu = (VNR['cpu'] - self._VNR['cpu']) / self._max_cpu
        bandwidth = (VNR['bandwidth'] - self._VNR['bandwidth']) / self._max_bandwidth
        saltos = (VNR['qtd_saltos'] - len(self._VNR['topologia'])) / self.substrato.number_of_edges() 
        
        if cpu >= 0 and bandwidth < 0:
            return bandwidth
        elif cpu < 0 and bandwidth >= 0:
            return cpu

        return cpu + bandwidth - saltos
    
    def calcular_pontuacao(self, populacao):
        return [self.fitness(x) for x in populacao]
    
    def calculo_roleta(self, pontuacoes):
        menor_pontuacao = min(pontuacoes)
        
        pontuacoes_positivadas = list(map(lambda pontuacao: pontuacao-menor_pontuacao, pontuacoes))
        
        maior_pontuacao = max(pontuacoes_positivadas)
        
        pontuacoes_positivadas_reverso = list(map(lambda pontuacao: (maior_pontuacao-pontuacao) * 5, pontuacoes_positivadas))

        roleta = []
        total = sum(pontuacoes_positivadas_reverso)
        for i, valor in enumerate(pontuacoes_positivadas_reverso):
            repeticao = round(abs((total+1)/(valor + 1)))
            
            roleta.extend([i] * repeticao)
      
        return roleta
    
    def crossover(self, populacao, g1, g2):
        # {'nos': [109, 105, 66], 'cpu': 0, 'bandwidth': 100, 'qtd_saltos': 5}
        qtd_nos = len(populacao[g1]['nos'])
        
        new_individuo = {'nos': [-1 for _ in range(qtd_nos)]}
        
        metade = int(np.ceil(qtd_nos/2))
        
        indices_dos_nos = list(range(qtd_nos))
        random.shuffle(indices_dos_nos)    
        
        if random.random() < 0.5:
            g1, g2 = g2, g1
        
        g = g1
        for i, indice in enumerate(indices_dos_nos):
            if i == metade:
                g = g2
                
            if populacao[g]['nos'][indice] not in new_individuo['nos']:
                new_individuo['nos'][indice] = populacao[g]['nos'][indice]
            else:
                nos_possiveis = list(set(range(self.substrato.number_of_nodes())).difference(set(new_individuo['nos'])))
                new_individuo['nos'][indice] = random.choice(nos_possiveis)

        return self.atualiza_individuo(new_individuo)
    
    def melhor_individuo(self, populacao):
            pontuacoes = self.calcular_pontuacao(populacao)
                  
            indice_maior_pontuacao = pontuacoes.index(max(pontuacoes))
            
            return populacao[indice_maior_pontuacao]
        
    def novo_individuo(self, populacao, prob_mutacao, roleta, param_para_paralelizar):
        genitor1 = random.choice(roleta)
        genitor2 = random.choice(roleta)
        filho = self.crossover(populacao, genitor1, genitor2)

        if (random.random() < prob_mutacao):
            qtd_nos_individuo = len(filho['nos'])
            qtd_nos_substrato = self.substrato.number_of_nodes()
                    
            indice_no = random.choice(range(qtd_nos_individuo))
                    
            new_nos_possiveis = list(set(range(qtd_nos_substrato)).difference(set(filho['nos'])))
                        
            filho['nos'][indice_no] = random.choice(new_nos_possiveis)

            filho = self.atualiza_individuo(filho)
            
        return filho
            
    def fit(self, repeticoes=100, prob_mutacao=0.1, verbose=False):
        populacao = list(map(self.atualiza_individuo, self._populacao_inicial.copy()))
            
        tam_pop = len(populacao) 

        for count in range(repeticoes):            
            if verbose:
                print("Repetição", count)
            
            with concurrent.futures.ThreadPoolExecutor(6) as executor:
                pontuacoes = list(executor.map(self.fitness, populacao))

                indice_maior_pontuacao = pontuacoes.index(max(pontuacoes))
                if verbose:
                    print('MELHOR INDIVIDUO DA EPOCA ', populacao[indice_maior_pontuacao], 'FITNESS: ', pontuacoes[indice_maior_pontuacao])

                roleta = self.calculo_roleta(pontuacoes)

                filhos = [populacao[indice_maior_pontuacao]]

                novo_individuo_partial = partial(self.novo_individuo, populacao, prob_mutacao, roleta)

                new_filhos = list(executor.map(novo_individuo_partial, range(tam_pop-1)))

                filhos += new_filhos
                        
            populacao = np.array(filhos) 
    
        return self.melhor_individuo(filhos), populacao
    
    def mapear_rede_virtual(self, request):
        print('Virtual Network requisitada: ', request, '\n')

        self.VNR = request

        melhor_individuo, populacao = self.fit(repeticoes=2000, prob_mutacao=0.2)

        return (melhor_individuo, request)
    
    def rede_virtual_valida(self, melhor_individuo, request):
        if (melhor_individuo['cpu'] >= request['cpu']) and (melhor_individuo['bandwidth'] >= request['bandwidth']):
            for no in melhor_individuo['nos']:
                self.substrato.nodes[no]['cpu'] -= request['cpu']

            for no1, no2 in request['topologia']:
                caminho = nx.dijkstra_path(self.substrato, melhor_individuo['nos'][no1], melhor_individuo['nos'][no2])

                for i, no in enumerate(caminho[1:]):
                    self.substrato.edges[(caminho[i], no)]['bandwidth'] -= request['bandwidth']

            print('Virtual Network mapeada: ', melhor_individuo, 'para a VNR: ', request, '\n')
        else:
            print('Não encontrou uma solução para a VNR ', request, '\n')

In [34]:
alg_genet = AlgoritmoGeneticoParalelo(G.copy(), None, populacao_inicial)

In [ ]:
t1 = process_time()

for request in new_vnrs:
    melhor_individuo, _ = alg_genet.mapear_rede_virtual(request)
    alg_genet.rede_virtual_valida(melhor_individuo, request)
    
t2 = process_time()

print('Tempo decorrido em segundos: ', t2-t1)

Virtual Network requisitada:  {'cpu': 17, 'bandwidth': 949, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2], [1, 2]]} 

Virtual Network mapeada:  {'nos': [97, 20, 104], 'cpu': 91, 'bandwidth': 4625, 'qtd_saltos': 8} para a VNR:  {'cpu': 17, 'bandwidth': 949, 'qtd_nos': 3, 'topologia': [[0, 1], [0, 2], [1, 2]]} 

Virtual Network requisitada:  {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]} 

Virtual Network mapeada:  {'nos': [9, 6, 10], 'cpu': 61, 'bandwidth': 7567, 'qtd_saltos': 3} para a VNR:  {'cpu': 22, 'bandwidth': 991, 'qtd_nos': 3, 'topologia': [[0, 2], [1, 2]]} 

Virtual Network requisitada:  {'cpu': 42, 'bandwidth': 1044, 'qtd_nos': 3, 'topologia': [[1, 2]]} 



## Teste visual com substrato de 10 nós

In [ ]:
del G

In [ ]:
G = nx.Graph()

# Determinando cpu como 1000 de modo que haja recurso suficiente para mapear todas as requisições
G.add_nodes_from([(no, {"cpu": 1000}) for no in range(10)])

G.nodes.data('cpu')

In [ ]:
G.add_edges_from([(no1, no2, {"bandwidth": 10000}) for no1 in range(9) for no2 in range(no1+1, 10)])

G.edges.data('bandwidth')

In [ ]:
import matplotlib.pyplot as plt

edge_labels = dict([((n1, n2), bandwidth) for n1, n2, bandwidth in G.edges.data('bandwidth')])
pos = nx.spring_layout(G)
plt.figure()

nx.draw(
    G, pos, edge_color='black', width=1, linewidths=1,
    node_size=500, node_color='pink', alpha=0.9,
    labels={node: cpu for node, cpu in G.nodes.data('cpu')}
)

nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=edge_labels,
    font_color='red'
)

plt.figure(figsize=(5,1))
plt.axis('off')
plt.show()

In [ ]:
with open('populacao_inical2.json', 'r') as f:
    populacao_inicial = json.load(f)

In [ ]:
alg_genet = AlgoritmoGenetico(G, None, populacao_inicial)

for request in new_vnrs:  
    melhor_individuo, _ = alg_genet.mapear_rede_virtual(request)
    alg_genet.rede_virtual_valida(melhor_individuo, request)

In [ ]:
G = alg_genet.substrato

import matplotlib.pyplot as plt

edge_labels = dict([((n1, n2), bandwidth) for n1, n2, bandwidth in G.edges.data('bandwidth')])
pos = nx.spring_layout(G)
plt.figure()

nx.draw(
    G, pos, edge_color='black', width=1, linewidths=1,
    node_size=500, node_color='pink', alpha=0.9,
    labels={node: cpu for node, cpu in G.nodes.data('cpu')}
)

nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=edge_labels,
    font_color='red'
)

plt.figure(figsize=(5,1))
plt.axis('off')
plt.show()